In [18]:
from ultralytics import YOLO
import json
import os

# Load a COCO-pretrained YOLO11n model
model = YOLO("yolo11n.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/parshvamehta/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [19]:
print(model)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [20]:
# Define paths
json_path = "annotations.json"  # JSON file path
images_dir = "images"  # Image folder
labels_dir = "labels"  # Output labels directory

os.makedirs(labels_dir, exist_ok=True)

# Load JSON annotation file
with open(json_path, "r") as f:
    data = json.load(f)

# Extract filenames and bounding boxes
file_names = data["train"]["file_names"]
rois_list = data["train"]["rois_list"]
occupancy_list = data["train"]["occupancy_list"]

for file_name, rois, occupancy in zip(file_names, rois_list, occupancy_list):
    label_file = os.path.join(labels_dir, file_name.replace(".JPG", ".txt"))

    with open(label_file, "w") as lf:
        for obj, occupied in zip(rois, occupancy):  # Match each bounding box to its occupancy status
            x_values = [p[0] for p in obj]  # Extract X-coordinates
            y_values = [p[1] for p in obj]  # Extract Y-coordinates

            x_center = sum(x_values) / len(x_values)
            y_center = sum(y_values) / len(y_values)
            width = max(x_values) - min(x_values)
            height = max(y_values) - min(y_values)

            class_id = 0 if occupied else 1  # Car = 0, Empty space = 1

            print(f"Image: {file_name}, Class: {class_id}, BBox: ({x_center}, {y_center}, {width}, {height})")  # Debug

            lf.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
    
    print(f"Image: {file_name}, Occupancy List: {occupancy}")
    print(f"Bounding Boxes: {len(rois)}, Occupancy Labels: {len(occupancy)}")

print("YOLO annotation conversion complete.")

Image: GOPR0025.JPG, Class: 0, BBox: (0.23870962499999998, 0.51505375, 0.18790325, 0.196774)
Image: GOPR0025.JPG, Class: 0, BBox: (0.4427419375, 0.6521505833333334, 0.3048384999999999, 0.2849463333333334)
Image: GOPR0025.JPG, Class: 0, BBox: (0.7151209374999999, 0.7997311666666667, 0.30564525, 0.2580646666666667)
Image: GOPR0025.JPG, Class: 1, BBox: (0.356048375, 0.31344083333333334, 0.17661300000000002, 0.10752666666666666)
Image: GOPR0025.JPG, Class: 0, BBox: (0.5336693125, 0.382258, 0.2403225, 0.160215)
Image: GOPR0025.JPG, Class: 0, BBox: (0.7471774375, 0.48333325000000005, 0.22499999999999998, 0.19247300000000006)
Image: GOPR0025.JPG, Class: 0, BBox: (0.9268144375, 0.5782258333333333, 0.14758074999999993, 0.17526866666666663)
Image: GOPR0025.JPG, Class: 0, BBox: (0.2444425, 0.12557266666666667, 0.09175000000000003, 0.07499999999999998)
Image: GOPR0025.JPG, Class: 0, BBox: (0.30672374999999996, 0.06686433333333333, 0.07100000000000001, 0.049)
Image: GOPR0025.JPG, Class: 0, BBox: (0

In [21]:
# Verify label assignments
for label_file in os.listdir(labels_dir):
    with open(os.path.join(labels_dir, label_file), "r") as f:
        content = f.readlines()
        print(f"{label_file}: {content}")  # Check class assignments

GOPR0024.txt: ['0 0.6232965 0.4611665833333333 0.2183447500000001 0.2272973333333333\n', '0 0.46978975 0.32709866666666665 0.17520825000000007 0.15066666666666667\n', '0 0.363353875 0.2417186666666667 0.1258685 0.09737533333333331\n', '0 0.5536128125 0.7507938333333333 0.27959100000000003 0.34481099999999987\n', '0 0.38090925 0.541023 0.20741325 0.2619693333333332\n', '1 0.2732259375 0.39293225 0.13652275000000003 0.1677843333333333\n', '0 0.2100574375 0.29942583333333334 0.09518750000000004 0.11425000000000005\n', '0 0.17433468750000003 0.24339625 0.06395825 0.06944433333333336\n', '1 0.1546545 0.21221566666666666 0.048541749999999995 0.04984700000000003\n', '1 0.14043825 0.18878166666666668 0.042416499999999996 0.04550000000000001\n', '1 0.1279924375 0.16812333333333335 0.03737499999999999 0.03883333333333333\n', '0 0.21879125 0.14100008333333333 0.054208249999999986 0.035416666666666666\n', '0 0.1967965 0.12600725000000002 0.04150000000000001 0.026444333333333347\n', '0 0.1803069375

In [22]:
# Count class instances
class_counts = {0: 0, 1: 0}  # Initialize counters
for file in os.listdir(labels_dir):
    with open(os.path.join(labels_dir, file), "r") as f:
        lines = f.readlines()
        for line in lines:
            class_id = int(line.split()[0])  # Extract class ID
            class_counts[class_id] += 1

print(f"Cars (0): {class_counts[0]}")
print(f"Empty Spaces (1): {class_counts[1]}")

Cars (0): 3940
Empty Spaces (1): 3902


In [23]:
# Validate data.yaml structure
with open("data.yml", "r") as f:
    print("\n[DATA.YML CONTENT]")
    print(f.read())


[DATA.YML CONTENT]
train: ./images  # Training data path
val: ./images  # No validation, but YOLO requires a path
nc: 2  # Number of classes (Car and Empty Space)
names: ["car", "empty"]


In [24]:
# Train with all images
# Train the model with adjusted class weights
data_yml_path = "./data.yml"
results = model.train(
    data=data_yml_path, 
    epochs=5, 
    imgsz=640
)

# Change path in C:\Users\[user]\AppData\Roaming\Ultralytics\settings.json to where the "ML Model" folder is stored on your computer

New https://pypi.org/project/ultralytics/8.3.81 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.80  Python-3.13.2 torch-2.6.0+cpu CPU (Intel Core(TM) i7-8700K 3.70GHz)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=./data.yml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train20, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sav

train: Scanning C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\labels.cache... 231 images, 62 backgrounds, 0 corrupt: 100%|██████████| 293/293 [00:00<?, ?it/s]
val: Scanning C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\labels.cache... 231 images, 62 backgrounds, 0 corrupt: 100%|██████████| 293/293 [00:00<?, ?it/s]


Plotting labels to runs\detect\train20\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train20
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5         0G      2.353      3.488      1.864        230        640: 100%|██████████| 19/19 [02:42<00:00,  8.54s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [01:11<00:00,  7.12s/it]

                   all        293       7842     0.0712      0.462      0.143     0.0626


...

        5/5         0G      1.926      1.694      1.414        255        640: 100%|██████████| 19/19 [01:54<00:00,  6.01s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:50<00:00,  5.01s/it]

                   all        293       7842      0.519      0.385      0.413      0.228



5 epochs completed in 0.275 hours.
Optimizer stripped from runs\detect\train20\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train20\weights\best.pt, 5.5MB

Validating runs\detect\train20\weights\best.pt...
Ultralytics 8.3.80  Python-3.13.2 torch-2.6.0+cpu CPU (Intel Core(TM) i7-8700K 3.70GHz)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:40<00:00,  4.05s/it]


                   all        293       7842      0.522      0.384      0.413      0.228
                   car        231       7842      0.522      0.384      0.413      0.228
Speed: 1.6ms preprocess, 57.3ms inference, 0.0ms loss, 23.9ms postprocess per image
Results saved to runs\detect\train20


In [ ]:
# Test inference with lower confidence threshold
image_path = "test_image.jpg"
results = model(image_path, conf=0.1)  # Lower confidence threshold to capture low-confidence detections
results.show()

# Visualize augmented training data to debug issues
model.train(data=data_yml_path, epochs=5, imgsz=640, visualize=True)

print("Training and debugging steps complete!")

FileNotFoundError: test_image.jpg does not exist